# IAU - Inteligentná analýza údajov (2024/2025)

#### Autori: Jan Lenhart (50 %), Marek Čederle (50 %)
##### Cvičenie: Pondelok 15:00, Cvičiaci: Ing. Oleksandr Lytvyn

<font color='salmon'>
    <b>Upozornenie:</b>
    Niektoré bloky kódu trvajú dlhú dobu ak ide o trénovanie modelov a podobne. (runtime celeho dokumentu ~30 min)
</font>

## Fáza 3 - Strojové učenie

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statistics import LinearRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, IsolationForest
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
from sklearn.tree import plot_tree, DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest, f_regression, f_classif, chi2, RFE, VarianceThreshold, SelectFromModel, mutual_info_regression
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, PowerTransformer, StandardScaler, RobustScaler, QuantileTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.linear_model import LogisticRegression


Na začiatok sme si importovali predspracované dáta z druhej fázy, s ktorými sme ďalej pracovali.

In [2]:
train = pd.read_csv('preprocessed_train.csv', delimiter=',')
test  = pd.read_csv('preprocessed_test.csv', delimiter=',')

In [3]:
X_train, X_test, y_train, y_test = train.drop(columns=['mwra']), test.drop(columns=['mwra']), train['mwra'], test['mwra']

In [4]:
X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

### 3.1 Jednoduchý klasifikátor na základe závislosti v dátach

#### 3.1.A Naimplementujte jednoduchý ID3 klasifikátor s hĺbkou min 2 (vrátane root/koreň).

```py
class CustomClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self):
        pass

    def fit(self, X, y):
        X, y = check_X_y(X, y)
        return self

    def predict(self, X):
        check_is_fitted(self)
        X = check_array(X)
        return np.zeros(X.shape[0])

    def score(self, X, y):
        y_pred = self.predict(X)
        return np.mean(y_pred == y)
```

Nasledujúci kód, predstavujúci vlastný ID3 klasifikátor sme implementovali z vyššie uvedeného kódu, ktorý bol generovaný pomocou ChatGPT. Daný kód predstavuje šablónu pre implementáciu vlastného klasifikátora knižnice `sklearn`.

In [5]:
# following code is co-engineered with ChatGPT
class ID3Classifier(BaseEstimator, ClassifierMixin):
    def __init__(self):
        self.tree_ = None

    def _entropy(self, y):
        counts = np.bincount(y)
        probabilities = counts / len(y)
        return -np.sum([p * np.log2(p) for p in probabilities if p > 0])

    def _information_gain(self, X, y, feature):
        total_entropy = self._entropy(y)
        values, counts = np.unique(X[:, feature], return_counts=True)
        weighted_entropy = sum(
            (counts[i] / sum(counts)) * self._entropy(y[X[:, feature] == values[i]])
            for i in range(len(values))
        )
        return total_entropy - weighted_entropy
    
    def _get_optimal_threshold(self, X, y, feature):
        values = np.unique(X[:, feature])
        best_threshold = None
        best_info_gain = -float('inf')
        for value in values:
            left_mask = X[:, feature] < value
            right_mask = ~left_mask
            left_y = y[left_mask]
            right_y = y[right_mask]
            
            left_entropy = self._entropy(left_y)
            right_entropy = self._entropy(right_y)
            
            weighted_entropy = (len(left_y) / len(y)) * left_entropy + (len(right_y) / len(y)) * right_entropy
            info_gain = self._entropy(y) - weighted_entropy
            
            if info_gain > best_info_gain:
                best_info_gain = info_gain
                best_threshold = value

        return best_threshold

    def _id3(self, X, y, features):
        if len(np.unique(y)) == 1:
            return {'value': np.unique(y)[0]}
        if len(features) == 0:
            return {'value': np.bincount(y).argmax()}
        best_feature = max(features, key=lambda f: self._information_gain(X, y, f))
        best_threshold = self._get_optimal_threshold(X, y, best_feature)
        tree = {'feature': best_feature, 'threshold': best_threshold, 'left': None, 'right': None}
        left_mask = X[:, best_feature] < best_threshold
        right_mask = ~left_mask
        left_X, left_y = X[left_mask], y[left_mask]
        right_X, right_y = X[right_mask], y[right_mask]
        tree['left'] = self._id3(left_X, left_y, [f for f in features if f != best_feature])
        tree['right'] = self._id3(right_X, right_y, [f for f in features if f != best_feature])
        return tree

    def fit(self, X, y):
        y = y.astype(int)
        X, y = check_X_y(X, y)
        self.tree_ = self._id3(X, y, range(X.shape[1]))
        return self
    
    def _predict_instance(self, instance, tree):
        if 'value' in tree:
            return tree['value']
        feature_value = instance[tree['feature']]
        if feature_value <= tree['threshold']:
            return self._predict_instance(instance, tree['left'])
        return self._predict_instance(instance, tree['right'])

    def predict(self, X):
        check_is_fitted(self, "tree_")
        X = check_array(X)
        return np.array([self._predict_instance(row, self.tree_) for row in X])

    # def score(self, X, y):
    #     y_pred = self.predict(X)
    #     return np.mean(y_pred == y)

Opis funkcií vlastného ID3 klasifikátora:

- `_entropy()`: Vypočíta entropiu podľa štandardného vzorca.

- `_information_gain()`: Vypočíta information gain podľa štandardného vzorca.

- `_get_optimal_threshold()`: Vypočíta optimálny threshold pre daný atribút (feature) na základe entropie a information gain.

- `_id3()`: Zo vstupných dát vytvorí ID3 rozhodovací strom, ktorý sa trénuje.

- `fit()`: Spústí trénovanie modelu.

- `_predict_instance()`: Predikuje výsledok jednej inštancie resp. jedného riadoku dát.

- `predict()`: Predikuje výsledné hodnoty na základe natrénovaného modelu a vstupených dát.

<!-- - `score()`: Vypočíta skóre resp. presnosť (accuracy) modelu. -->



In [ ]:
# avg runtime ~1 min
id3_classifier = ID3Classifier()
id3_classifier.fit(X_train, y_train)

**Jednoduchá CLI Style vizualizácia nášho ID3 rozhodovacieho stromu**

In [7]:
def id3_print_recursive(tree, X, y, depth = 0, left = False, right = False):
    for i in range(depth):
        print(end='  ')
    if (left):
        print('l-', end='')
    if (right):
        print('r-', end='')
    if ('feature' in tree):
        print(f"feature: {X.columns[tree['feature']]}, threshold: {tree['threshold']}")
        if (tree['left']):
            id3_print_recursive(tree['left'], X, y, depth + 1, left=True)
        if (tree['right']):
            id3_print_recursive(tree['right'], X, y, depth + 1, right=True)
    else:
        print(f"value: {tree['value']}")

In [ ]:
print(id3_classifier.tree_)

In [ ]:
id3_print_recursive(id3_classifier.tree_, X_train, y_train)

#### 3.1.B Vyhodnoťte Váš ID3 klasifikátor pomocou metrík accuracy, precision a recall.

In [10]:
def print_eval(y, y_pred):
    accuracy = accuracy_score(y, y_pred)
    precision = precision_score(y, y_pred, average='weighted')
    recall = recall_score(y, y_pred, average='weighted')

    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")

In [ ]:
# Test data
id3_y_pred_test = id3_classifier.predict(X_test)
print_eval(y_test, id3_y_pred_test)

Môžeme vidieť že všetky naše metriky dávajú rovnakú hodnotu. Zároveň môžeme konštatovať, že úspešnosť `82%` nášho klasifikátora nie je zlá, ale ani nie je ideálna resp. nejde o najlepší machine learning model na základe naších dát.

#### 3.1.C Zístite či Váš ID3 klasifikátor má overfit.

In [12]:
id3_y_pred_train = id3_classifier.predict(X_train)
id3_y_pred_train = pd.DataFrame(id3_y_pred_train, columns=['mwra'])

In [ ]:
# Train data
print_eval(y_train, id3_y_pred_train)

In [ ]:
# Test data
print_eval(y_test, id3_y_pred_test)

Rozdiel (delta) medzi trénovacou a testovacou množinou je `3%`, čo znamená, že náš model nemá overfit a celková úspešnosť je relatívne vysoká, čo značí že náš model nemá ani underfit.

### 3.2 Trénovanie a vyhodnotenie klasifikátorov strojového učenia

#### 3.2.A Na trénovanie využite jeden stromový algoritmus v scikit-learn.

Ako stromový algoritmus sme si zvolili `RandomForestClassifier`.

In [15]:
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1)

In [ ]:
rf_model.fit(X_train, y_train)

In [ ]:
# Train data
rf_y_pred = rf_model.predict(X_train)

print_eval(y_train, rf_y_pred)

In [ ]:
# Test data
rf_y_pred = rf_model.predict(X_test)

print_eval(y_test, rf_y_pred)

#### 3.2.B Porovnajte s jedným iným nestromovým algoritmom v scikit-learn.

Ako nestromový algoritmus sme si zvolili `SVC` (Support Vector Classifier).

In [19]:
svc_model = SVC(kernel='rbf', random_state=42)

In [ ]:
svc_model.fit(X_train, y_train)

In [ ]:
# Train data
svc_y_pred = svc_model.predict(X_train)

print_eval(y_train, svc_y_pred)

In [ ]:
# Test data
svc_y_pred = svc_model.predict(X_test)

print_eval(y_test, svc_y_pred)

#### 3.2.C Porovnajte výsledky s ID3 z prvého kroku.

Pre ukážku sme si vyskúšali ešte aj `DecisionTreeClassifier` klasifikátor, aby sme mali viacej modelov na porovnanie.

In [ ]:
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

In [ ]:
# Train data
print_eval(y_train, dt_model.predict(X_train))

In [ ]:
# Test data
dt_y_pred = dt_model.predict(X_test)
print_eval(y_test, dt_y_pred)

Úspešnosť ID3 klasifikátora na testovacej množine je v porovnaní o `2%` menšia ako úspešnosť `RandomForestClassifier` a `SVC` klasifikátora. Na druhej strane je o `14%` lepšia ako `DecisionTreeClassifier`.

Zároveň si môžeme všimnúť, že `ID3` a `SVC` modely nemajú ani underfit ani overfit, zatiaľ čo `RandomForestClassifier` a `DecisionTreeClassifier` majú overfit čo znamená, že nedokážu generalizovať na testovacej množine.

#### 3.2.D Vizualizujte natrénované pravidlá minimálne pre jeden Vami vybraný algoritmus

Rozhodli sme sa vizualiovať pravidlá pre `RandomForestClassifier` klasifikátor, pretože ide o veľký model, ktorý má viacej stromov, tak sme vizualizovali jeden čo je na nultom indexe (prvý strom v poradí).

In [ ]:
# avg runtime ~35 sec
plt.figure(figsize=(12, 8))
plot_tree(rf_model.estimators_[0], feature_names=X.columns, class_names=X.columns, filled=True)
plt.title("Vizualizácia stromu na indexe č.0 RandomForest modelu")
plt.show()

Následne sme si urobili aj krajšiu vizualizáciu nášho `ID3` klasifikátora, ktorý je jednoduchší ako `RandomForest` a má len niekoľko pravidiel.

In [27]:
def id3_plot_tree(tree, x=0.5, y=1, width=0.2, ax=None, feature_names=None, depth=0):
    if ax is None:
        fig, ax = plt.subplots(figsize=(12, 8))
    ax.axis('off')

    if 'value' in tree:
        ax.text(x, y, f'mwra: {tree["value"]}', ha='center', va='center', 
                bbox=dict(facecolor='lightgray', edgecolor='black', boxstyle='round,pad=1'))
        return ax

    feature_name = feature_names[tree['feature']]
    text = f'{feature_name} <= {tree["threshold"]:.2f}' if tree['threshold'] else feature_name

    ax.text(x, y, text, ha='center', va='center', bbox=dict(facecolor='lightblue', edgecolor='black', boxstyle='round,pad=1'))

    left_x = x - width / 2
    right_x = x + width / 2
    child_y = y - 1 / (depth + 1)

    if (tree['left']):
        ax.plot([x, left_x], [y, child_y], 'k-')
        id3_plot_tree(tree['left'], x=left_x, y=child_y, width=width / 2, ax=ax, feature_names=feature_names, depth=depth + 1)
    if (tree['right']):
        ax.plot([x, right_x], [y, child_y], 'k-')
        id3_plot_tree(tree['right'], x=right_x, y=child_y, width=width / 2, ax=ax, feature_names=feature_names, depth=depth + 1)

    return ax

In [ ]:
fig, ax = plt.subplots(figsize=(40, 8))
id3_plot_tree(id3_classifier.tree_, feature_names=X.columns, ax=ax)
plt.show()

#### 3.2.E Vyhodnoťte natrénované modely pomocou metrík accuracy, precision a recall

Vyhodnotenie metrík pre všetky modeli bolo spomenuté už vyššie v sekcii `3.2.C`, keď sme porovnávali ich úspešnosť a prítomnosť overfitu resp. underfitu.

### 3.3 Optimalizácia alias hyperparameter tuning

#### 3.3.A Vyskúšajte rôzne nastavenie hyperparametrov (tuning) pre zvolený algoritmus tak, aby ste optimalizovali výkonnosť (bez underfitingu).

Keďže `RandomForestClassifier` model mal overfit, tak sme si ho vybrali ako kandidáta na optimalizáciu pomocou hyperparameter tuningu.

In [ ]:
# avg runtime
# n_jobs=-1 ->  ~4 min

model_new = RandomForestClassifier(random_state=42, n_jobs=-1)

param_grid_new = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5]
}

grid_search_new = GridSearchCV(model_new, param_grid_new, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_new.fit(X_train, y_train)

tuning_model_new = grid_search_new.best_estimator_

print("Best Parameters:", grid_search_new.best_params_)
print("Best Score:", grid_search_new.best_score_)

In [ ]:
print("Tuning Model (RandomForest) Train Score:", tuning_model_new.score(X_train, y_train))
print("Tuning Model (RandomForest) Test  Score:", tuning_model_new.score(X_test, y_test))

Môžeme si všimnúť, že model po optimalizácii nemá overfit, pretože delta medzi úspešnosťou trénovacej a testovacej množiny je `3%`, čo je výrazné zlepšenie oproti pôvodným `11%`. A ani underfit pretože úspešnosť je relatívne vysoká.

Zároveň sme si vyskúšali hyperparameter tuning aj pre `SVC` model, ktorý nemal overfit.

In [ ]:
# avg runtime ~
# n_jobs=-1 -> 3 min

model = SVC()

param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf'],
    'gamma': [0.01, 0.1, 1, 10]
    # 'C': [10, 100, 1000],
    # 'kernel': ['linear', 'rbf', 'poly'],
    # 'gamma': [0.1, 0.01, 0.001]
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

tuning_model = grid_search.best_estimator_

print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

In [ ]:
print("Tuning Model (SVC) Train Score:", tuning_model.score(X_train, y_train))
print("Tuning Model (SVC) Test  Score:", tuning_model.score(X_test, y_test))

Môžeme vidieť, že jeho úspešnosť sa nijako nezmenila s tým, že model nemá underfit ani overfit.

#### 3.3.B Vyskúšajte kombinácie modelov (ensemble) pre zvolený algoritmus tak, aby ste optimalizovali výkonnosť (bez underfitingu).

Ako modely do ensemble sme si zvolili `RandomForestClassifier` a `SVC`, ktoré sme následne skombinovali do jedného modelu pomocou `StackingClassifier`. Ako parametre modelov sme určili tie, ktoré sme získali z hyperparameter tuning.

In [33]:
# avg runtime
# n_jobs=-1 -> 40 sec

base_models = [
    ('svm', SVC(C=10, gamma=0.1, kernel='rbf', probability=True)),
    ('tree', RandomForestClassifier(max_depth=10, min_samples_leaf=5, min_samples_split=2, n_estimators=200, n_jobs=-1))
]

stacking_model = StackingClassifier(estimators=base_models, final_estimator=LogisticRegression(), n_jobs=-1)
stacking_model.fit(X_train, y_train)

ensemble_model = stacking_model

In [ ]:
print("Enseble Model Train Score:", ensemble_model.score(X_train, y_train))
print("Enseble Model Test  Score:", ensemble_model.score(X_test, y_test))

Môžeme vidieť že úspešnosť modelu je `84%` na testovacej množine, čo značí o tom, že model nemá underfit. Bohužiaľ sa nám stále nedarí optimalizovať model tak, aby mal vyššiu úspešnosť.

#### 3.3.C Využite krížovú validáciu (cross validation) na trénovacej množine.

Najskôr sme si urobili krížovú validáciu pre `RandomForestClassifier` model, ktorý sme optimalizovali pomocou parametrov ktoré nám vrátily `GridSearchCV` vyššie v dokumente.

In [ ]:
# avg runtime ~50 sec
# n_jobs=-1 -> 6 sec

model = RandomForestClassifier(max_depth=10, min_samples_leaf=5, min_samples_split=2, n_estimators=200, n_jobs=-1)

cv_scores = cross_val_score(model, X_train, y_train, cv=10, scoring='accuracy', n_jobs=-1)
print("Cross-Validation Scores:", cv_scores)
print("Mean CV Score:", cv_scores.mean())

Následne sme si urobili krížovú validáciu pre `SVC` model, ktorý sme tiež pomocou `GridSearchCV` optimalizovali.

In [ ]:
# avg runtime ~25 sec
# n_jobs=-1 -> 4 sec
cv_scores = cross_val_score(tuning_model, X_train, y_train, cv=10, scoring='accuracy', n_jobs=-1)
print("Cross-Validation Scores:", cv_scores)
print("Mean CV Score:", cv_scores.mean())

Ako poslednú sme si vyskúšali krížovú validáciu pre `StackingClassifier` model, ktorý sme vytvorili z kombinácie `RandomForestClassifier` a `SVC` modelov.

In [ ]:
# avg runtime ~7,5 min
# n_jobs=-1 -> 1,5 min
cv_scores = cross_val_score(ensemble_model, X_train, y_train, cv=10, scoring='accuracy', n_jobs=-1)
print("Cross-Validation Scores:", cv_scores)
print("Mean CV Score:", cv_scores.mean())

Môžeme si všimnúť že všetky 3 druhy krížovej validácie majú rovnakú úspešnosť.

#### 3.3.D Dokážte že Váš nastavený najlepší model je bez overfitingu.

Podľa predchádzajúcich metrík sme zistili, že náš doposial najlepší model je optimalizovaný `SVC` model pomocou hyperparameter tuningu.

In [ ]:
print("Tuning Model Train Score:", tuning_model.score(X_train, y_train))
print("Tuning Model Test  Score:", tuning_model.score(X_test, y_test))

Podľa metriky `score` (`accuracy`) môžeme vidieť, že model nemá overfit, pretože delta medzi trénovacou a testovacou množinou je `~1%` a zároveň ani underfit pretože má relatívne vysokú úspešnosť.

### 3.4 Vyhodnotenie vplyvu zvolenej stratégie riešenia na klasifikáciu

Aby sme vyhodnotili, či náš proces predspracovania a modelovania bol správny, urobili sme bruteforce medzi veľa kombináciami objektov z predprocesing pipeline.

Urobili sme viacej iterácií bruteforce, kde po každej iterácii sme ju zanalyzovali a podľa nej sme navrhli ako má vyzerať ďalšia iterácia.

Na začiatku sme si načítali "raw" data a urobili sme si prípavu na robenie s pipelinami.

In [39]:
df_connections = pd.read_csv('112/connections.csv', delimiter=',')
df_processes   = pd.read_csv('112/processes.csv', delimiter=',')

merged_data = df_connections.merge(df_processes, on=['imei', 'ts'], how='inner')
merged_data['ts'] = pd.to_numeric(pd.to_datetime(merged_data['ts'], errors='coerce'))
merged_data = merged_data.drop_duplicates()
merged_data = merged_data.drop(columns=['mwra_y'])
merged_data = merged_data.rename(columns={"mwra_x": "mwra"})

X_raw = merged_data.drop(columns='mwra')
y_raw = merged_data['mwra']

X_raw_train, X_raw_test, y_raw_train, y_raw_test = train_test_split(X_raw, y_raw, test_size=0.2, random_state=42)

In [40]:
# co-engineered with ChatGPT - from phase 2
class OutlierHandler(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=1.5, outlier_fraction=0.05):
        self.threshold = threshold
        self.outlier_fraction = outlier_fraction
        self.imputer = SimpleImputer(strategy='median')

    def fit(self, X, y=None):
        self.iqr = np.percentile(X, 75, axis=0) - np.percentile(X, 25, axis=0)
        self.lower_bound = np.percentile(X, 25, axis=0) - self.threshold * self.iqr
        self.upper_bound = np.percentile(X, 75, axis=0) + self.threshold * self.iqr
        return self

    def transform(self, X):
        outliers_mask = np.any((X < self.lower_bound) | (X > self.upper_bound), axis=1)
        num_outliers = np.sum(outliers_mask)
        total_samples = X.shape[0]
        outlier_fraction = num_outliers / total_samples
        
        if outlier_fraction < self.outlier_fraction:
            return X[~outliers_mask]
        else:
            X_imputed = X.copy()
            X_imputed[outliers_mask] = np.nan
            return self.imputer.fit_transform(X_imputed)

Takto vyzeral náš pipeline na konci 2 fázy:
```py
preprocessor_pipeline = 
Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('outlier_handler', OutlierHandler(threshold=1.5, outlier_fraction=0.05)),
        ('scaler', MinMaxScaler()),
        ('power_transformer', PowerTransformer(method='yeo-johnson')),
        ('feature_selection', SelectKBest(mutual_info_regression, k=5))],
    verbose=True
)
```

Typy objektov, ktorých kombinácie sme skúšali v našom bruteforce:

In [41]:
# we commented out the ones with worst scores so the broteforce doesnt run so long

imputers = [
    SimpleImputer(strategy='mean'),
    SimpleImputer(strategy='median'),
    # KNNImputer(n_neighbors=5)
]

outlier_handlers = [
    OutlierHandler(threshold=1.5, outlier_fraction=0.05),
    OutlierHandler(threshold=2.0, outlier_fraction=0.1)
]

scalers = [
    MinMaxScaler(),
    # StandardScaler(),
    RobustScaler()
]

transformers = [
    PowerTransformer(method='yeo-johnson'),
    QuantileTransformer(output_distribution='normal'),
    # QuantileTransformer(output_distribution='uniform')
]

feature_selections = [
    SelectKBest(mutual_info_regression, k=5),
    SelectKBest(f_regression, k=5),
    # SelectKBest(f_classif, k=5),
    # SelectKBest(chi2, k=5),
    VarianceThreshold(threshold=0.1)
]

In [42]:
classifiers = [
    svc_model,
    tuning_model,
]

In [43]:
# avg runtime ~9,5 min
# 45 min last run
evaluated_models = []
good_preprocessor_count = 0
good_classifier_count = 0

def bruteforce(imputers, outlier_handlers, scalers, transformers, feature_selections, classifiers):
    global evaluated_models, good_preprocessor_count, good_classifier_count
    evaluated_models = []
    good_preprocessor_count = 0
    good_classifier_count = 0

    preprocessors = []
    for imputer in imputers:
        for outlier_handler in outlier_handlers:
            for scaler in scalers:
                for transformer in transformers:
                    for feature_selection in feature_selections:
                        preprocessor = Pipeline(steps=[
                            ('imputer', imputer),
                            ('outlier_handler', outlier_handler),
                            ('scaler', scaler),
                            ('transformer', transformer),
                            ('feature_selection', feature_selection)
                            ],
                            verbose=False
                        )
                        preprocessors.append(preprocessor)

    model_index = 0
    best_train_score = 0
    best_test_score = 0
    for preprocessor in preprocessors:
        print(f"preprocessor {preprocessors.index(preprocessor)+1}/{len(preprocessors)}")
        try:
            preprocessor.fit(X_raw_train, y_raw_train)

            X_train_preprocessed = preprocessor.transform(X_raw_train)
            X_train_preprocessed = pd.DataFrame(X_train_preprocessed, columns=X_raw_train.columns[preprocessor.named_steps['feature_selection'].get_support()], index=X_raw_train.index)

            X_test_preprocessed = preprocessor.transform(X_raw_test)
            X_test_preprocessed = pd.DataFrame(X_test_preprocessed, columns=X_raw_test.columns[preprocessor.named_steps['feature_selection'].get_support()], index=X_raw_test.index)

            for classifier in classifiers:
                print(f"PREPROCESSOR GOOD: {preprocessor}")
                good_preprocessor_count += 1
                
                try:
                    classifier.fit(X_train_preprocessed, y_raw_train)

                    train_score = classifier.score(X_train_preprocessed, y_train)
                    test_score = classifier.score(X_test_preprocessed, y_test)

                    if train_score > best_train_score or test_score > best_test_score:
                        model = Pipeline([
                            ('preprocessor', preprocessor),
                            ('classifier', classifier),
                            ],
                            verbose=False
                        )
                        
                        delta = train_score - test_score
                        y_pred = classifier.predict(X_test_preprocessed)
                        accuracy = accuracy_score(y_raw_test, y_pred)
                        precision = precision_score(y_raw_test, y_pred, average='weighted')
                        recall = recall_score(y_raw_test, y_pred, average='weighted')
                        
                        evaluated_models.append({
                            "train": train_score,
                            "test": test_score,
                            "delta": delta,
                            "index": model_index,
                            "model": model,
                            "accuracy": accuracy,
                            "precision": precision,
                            "recall": recall,
                        })

                        if train_score > best_train_score:
                            best_train_score = train_score
                        if test_score > best_test_score:
                            best_test_score = test_score

                    model_index += 1
                    
                    print(f"CLASSIFIER GOOD: {(classifier)}")
                    good_classifier_count += 1

                except Exception as ee:
                    print(f"CLASSIFIER BAD : {classifier}\n{ee}")
                    print("--------------------------------------------------------------------------------------------------")
                    model_index += 1
                
                print("--------------------------------------------------------------------------------------------------")

        except Exception as e:
            print(f"PREPROCESSOR BAD : {preprocessor}\n{e}")
            model_index += 2
            print("--------------------------------------------------------------------------------------------------")

In [ ]:
bruteforce(imputers, outlier_handlers, scalers, transformers, feature_selections, classifiers)

In [ ]:
print(f"good_preprocessor_count {good_preprocessor_count}")
print(f"good_classifier_count   {good_classifier_count}")
print(f"len(evaluated_models)   {len(evaluated_models)}")

In [46]:
def print_iteration_info_type_1():
    global evaluated_models
    evaluated_models = sorted(evaluated_models, key=lambda x: x['test'], reverse=True)
    for i in range(len(evaluated_models)):
        model = evaluated_models[i]
        print('train   test    delta  accuracy  precision  recall')
        print(f"{model['train']*100:.2f}%  ", end='')
        print(f"{model['test']*100:.2f}%  ", end='')
        print(f"{model['delta']*100:.2f}%  ", end='')
        print(f"{model['accuracy']*100:.2f}%    ", end='')
        print(f"{model['precision']*100:.2f}%     ", end='')
        print(f"{model['recall']*100:.2f}%  ")
        print(f"preprocessor:{model['model'].steps[0]}")
        print(f"classifier:{model['model'].steps[1]}")
        print('-------------------------------------------------------------------------------------------------------------------')

In [47]:
def print_iteration_info_type_2():
    global evaluated_models
    evaluated_models = sorted(evaluated_models, key=lambda x: x['test'], reverse=True)
    for step_i in range(len(evaluated_models[0]['model'].steps[0][1].steps)):
        for i in range(len(evaluated_models)):
            model = evaluated_models[i]
            print('train   test    delta  accuracy  precision  recall')
            print(f"{model['train']*100:.2f}%  ", end='')
            print(f"{model['test']*100:.2f}%  ", end='')
            print(f"{model['delta']*100:.2f}%  ", end='')
            print(f"{model['accuracy']*100:.2f}%    ", end='')
            print(f"{model['precision']*100:.2f}%     ", end='')
            print(f"{model['recall']*100:.2f}%  ")
            print(model['model'].steps[0][1].steps[step_i])
            print()
        print('--------------------------------------------------------------------')

In [48]:
def print_iteration_info_type_3():
    global evaluated_models
    evaluated_models = sorted(evaluated_models, key=lambda x: x['delta'], reverse=False)
    for step_i in range(len(evaluated_models[0]['model'].steps[0][1].steps)):
        for i in range(len(evaluated_models)):
            model = evaluated_models[i]
            print('train   test    delta  accuracy  precision  recall')
            print(f"{model['train']*100:.2f}%  ", end='')
            print(f"{model['test']*100:.2f}%  ", end='')
            print(f"{model['delta']*100:.2f}%  ", end='')
            print(f"{model['accuracy']*100:.2f}%    ", end='')
            print(f"{model['precision']*100:.2f}%     ", end='')
            print(f"{model['recall']*100:.2f}%  ")
            print(model['model'].steps[0][1].steps[step_i])
            print()
        print('--------------------------------------------------------------------')

In [ ]:
print_iteration_info_type_1()

In [ ]:
print_iteration_info_type_2()

In [ ]:
print_iteration_info_type_3()

Optimal model from iteration 1:

```py
train: 89.03%  test:87.73%  delta:1.30%  
preprocessor:('preprocessor', Pipeline(steps=[('imputer', SimpleImputer()),
                ('outlier_handler',
                 OutlierHandler(outlier_fraction=0.1, threshold=2.0)),
                ('scaler', MinMaxScaler()), ('transformer', PowerTransformer()),
                ('feature_selection', VarianceThreshold(threshold=0.1))]))
classifier:('classifier', SVC(kernel='linear', random_state=42))
```

____________________________________________________________________________________
**Iteration 2**

In [52]:
imputers2 = [
    SimpleImputer(),
]

outlier_handlers2 = [
    OutlierHandler(threshold=2.0, outlier_fraction=0.1)
]

scalers2 = [
    MinMaxScaler(),
    RobustScaler()
]

transformers2 = [
    PowerTransformer(method='yeo-johnson'),
]

feature_selections2 = [
    SelectKBest(mutual_info_regression, k=5),
    SelectKBest(f_regression, k=5),
    VarianceThreshold(threshold=0.1)
]

In [53]:
classifiers2 = [
    svc_model,
    tuning_model,
]

In [ ]:
bruteforce(imputers2, outlier_handlers2, scalers2, transformers2, feature_selections2, classifiers2)

In [ ]:
print_iteration_info_type_1()

In [ ]:
print_iteration_info_type_2()

In [ ]:
print_iteration_info_type_3()

Optimal model from iteration 2:

train   test    delta

91.85%  90.20%  1.65%

```py
preprocessor:('preprocessor', Pipeline(steps=[('imputer', SimpleImputer()),
                ('outlier_handler',
                 OutlierHandler(outlier_fraction=0.1, threshold=2.0)),
                ('scaler', MinMaxScaler()), ('transformer', PowerTransformer()),
                ('feature_selection', VarianceThreshold(threshold=0.1))]))
classifier:('classifier', SVC(random_state=42))
```

In [58]:
imputers3 = [
    SimpleImputer(),
]

outlier_handlers3 = [
    OutlierHandler(threshold=2.0, outlier_fraction=0.1)
]

scalers3 = [
    MinMaxScaler(),
    RobustScaler()
]

transformers3 = [
    PowerTransformer(method='yeo-johnson'),
]

feature_selections3 = [
    SelectKBest(mutual_info_regression, k=7),
    SelectKBest(mutual_info_regression, k=9),
    SelectKBest(mutual_info_regression, k=11),
    SelectKBest(mutual_info_regression, k=13),
]

In [59]:
classifiers3 = [
    tuning_model,
]

In [ ]:
bruteforce(imputers3, outlier_handlers3, scalers3, transformers3, feature_selections3, classifiers3)

In [ ]:
print_iteration_info_type_1()

In [ ]:
print_iteration_info_type_2()

In [ ]:
print_iteration_info_type_3()

Optimal model from iteration 3:

train   test    delta

91.85%  90.10%  1.75%

```py
preprocessor:('preprocessor', Pipeline(steps=[('imputer', SimpleImputer()),
                ('outlier_handler',
                 OutlierHandler(outlier_fraction=0.1, threshold=2.0)),
                ('scaler', MinMaxScaler()), ('transformer', PowerTransformer()),
                ('feature_selection',
                 SelectKBest(k=11,
                             score_func=<function mutual_info_regression at 0x0000015DD0FB5800>))]))
classifier:('classifier', SVC(C=10, gamma=0.1))
```

In [64]:
imputers4 = [
    SimpleImputer(),
]

outlier_handlers4 = [
    OutlierHandler(threshold=2.5, outlier_fraction=0.1),
    OutlierHandler(threshold=2.5, outlier_fraction=0.05),
    OutlierHandler(threshold=3.0, outlier_fraction=0.1),
    OutlierHandler(threshold=3.0, outlier_fraction=0.05),
]

scalers4 = [
    MinMaxScaler(),
    RobustScaler()
]

transformers4 = [
    PowerTransformer(method='yeo-johnson'),
]

feature_selections4 = [
    SelectKBest(mutual_info_regression, k=11),
]

In [65]:
classifiers4 = [
    tuning_model,
]

In [ ]:
bruteforce(imputers4, outlier_handlers4, scalers4, transformers4, feature_selections4, classifiers4)

In [ ]:
print_iteration_info_type_1()

In [ ]:
print_iteration_info_type_2()

In [ ]:
print_iteration_info_type_3()

Optimal model from iteration 4:

train   test    delta

92.33%  90.67%  1.66%

```py
preprocessor:('preprocessor', Pipeline(steps=[('imputer', SimpleImputer()),
                ('outlier_handler', OutlierHandler(threshold=3.0)),
                ('scaler', RobustScaler()), ('transformer', PowerTransformer()),
                ('feature_selection',
                 SelectKBest(k=11,
                             score_func=<function mutual_info_regression at 0x0000015DD0FB5800>))]))
classifier:('classifier', SVC(C=10, gamma=0.1))
```

____________________________________________________________________________________
**Iteration 5**

In [70]:
# we commented out the ones with worst scores so the broteforce doesnt run so long

imputers5 = [
    SimpleImputer(strategy='mean'),
    # SimpleImputer(strategy='median'),
    # KNNImputer(n_neighbors=5)
]

outlier_handlers5 = [
    # OutlierHandler(threshold=1.5, outlier_fraction=0.05),
    OutlierHandler(threshold=2.0, outlier_fraction=0.1),
    OutlierHandler(threshold=3.0, outlier_fraction=0.05)
]

scalers5 = [
    MinMaxScaler(),
    # StandardScaler(),
    # RobustScaler()
]

transformers5 = [
    PowerTransformer(method='yeo-johnson'),
    # QuantileTransformer(output_distribution='normal'),
    # QuantileTransformer(output_distribution='uniform')
]

feature_selections5 = [
    SelectKBest(mutual_info_regression, k=5),
    SelectKBest(f_regression, k=5),
    SelectKBest(mutual_info_regression, k=11),
    SelectKBest(f_regression, k=11),
    # SelectKBest(f_classif, k=5),
    # SelectKBest(chi2, k=5),
    VarianceThreshold(threshold=0.1)
]

In [71]:
classifiers5 = [
    # DecisionTreeClassifier(random_state=42),
    # RandomForestClassifier(random_state=42),
    # svc_model,
    # tuning_model,
    tuning_model_new,
]

In [ ]:
bruteforce(imputers5, outlier_handlers5, scalers5, transformers5, feature_selections5, classifiers5)

In [ ]:
print(f"good_preprocessor_count {good_preprocessor_count}")
print(f"good_classifier_count   {good_classifier_count}")
print(f"len(evaluated_models)   {len(evaluated_models)}")

In [ ]:
print_iteration_info_type_1()

In [ ]:
print_iteration_info_type_2()

In [ ]:
print_iteration_info_type_3()

Optimal model from iteration 5:

train   test    delta

92.31%  89.83%  2.48%

```py
preprocessor:('preprocessor', Pipeline(steps=[('imputer', SimpleImputer()),
                ('outlier_handler', OutlierHandler(threshold=3.0)),
                ('scaler', MinMaxScaler()), ('transformer', PowerTransformer()),
                ('feature_selection', VarianceThreshold(threshold=0.1))]))
classifier:('classifier', RandomForestClassifier(max_depth=10, min_samples_leaf=5, n_estimators=200,
                       n_jobs=-1, random_state=42))
```

#### 3.4.A Stratégie riešenia chýbajúcich hodnôt a outlierov

Chýbajúce hodnoty sme vždy dokázali nahradiť použitím `SimpleImputer`, pričom stačilo použiť stratégiu mean (priemer).
Najlepšie riešenie outlierov bolo, že sme zvýšili prahovú hodnotu (threshold). Ak dáta mali pár outlierov, tak model lepšie generalizoval.

#### 3.4.B Dátová transformácia (scaling, transformer, …)

Pri použití `RobustScaler` a `MinMaxScaler` boli výsledky modelov na podobnej úrovni, ale najlepšie modely využívali `MinMaxScaler`. `StandardScaler` takmer žiadny model nepoužil.


#### 3.4.C Výber atribútov, výber algoritmov, hyperparameter tuning, ensemble learning

Náš pôvodný výber príznakov (feature selection) bol k = 5, ale keď sme ho zvýšili na k = 11, zlepšila sa úspešnosť modelu. Pri väčších hodnotách `k` už ku zlepšeniu nedošlo.

#### 3.4.D Ktorý model je Váš najlepší model pre nasadenie (deployment)?

Z bruteforce sme zistili, že nasledujúci pipeline a model resp. klasifikátor je najlepší pre nasadenie do produkcie (deployment):

| Train   | Test   | Delta   | Accuracy | Precision | Recall |
|---------|--------|---------|----------|-----------|--------|
| 92.30%  | 90.70% | 1.60%   | 90.70%   | 90.70%    | 90.70% |


```py
preprocessor:('preprocessor', Pipeline(steps=[('imputer', SimpleImputer()),
                ('outlier_handler', OutlierHandler(threshold=3.0)),
                ('scaler', MinMaxScaler()), ('transformer', PowerTransformer()),
                ('feature_selection',
                 SelectKBest(k=11,
                             score_func=<function mutual_info_regression at 0x000001B52CA73100>))]))
classifier:('classifier', SVC(C=10, gamma=0.1))
```

#### 3.4.E Aký je data pipeline pre jeho vybudovanie na základe Vášho datasetu v produkcii?

Okrem modelu sme vyššie v sekcii `3.4.D` už spomenuli aj pipeline, ktorý sme použili na vytvorenie nášho najlepšieho modelu. Tento pipeline by sme mohli použiť aj v produkcii, pretože je najlepší zo všetkých, ktoré sme skúšali v bruteforce.

### Záver

Naše lepšie modeli v sekciách `3.1` až `3.3` boli relatívne dobré, pretože väčšinou nemali ani underfit ani overfit a boli stabilné, čo sme zistili z metrík accuracy, precision a recall. Avšak nemali takú vysokú úspešnosť ako sme si nazačiatku predstavovali. Neboli sme vlastne spokojný s úspešnosťou modelov, čoho príčinou bolo asi nie optimálne predspracovanie v druhej fáze. Preto sme sa rozhodli, že pomocou nejakej inteligentnej kombinácie bruteforcu, analýzy a greedy algoritmov zostrojíme najlepší pipeline a zároveň klasifikačný model, ktorý bude mať najlepšiu úspešnosť a nebude mať ani underfit ani overfit, čo sa nám vlastne podarilo.

Z pôvodných modelov, ktoré mali úspešnosť `~83%` sa nám podarilo vytvoriť model, ktorý má úspešnosť `~91%`, čo je veľký rozdiel a zlepšenie.

## Zdroje

Prednášky a cvičenia z predmetu IAU.

[IAU Github repozitár](https://github.com/FIIT-IAU/IAU-course)

[Scipy dokumentácia](https://docs.scipy.org/doc/scipy/reference/index.html)

[Numpy dokumentácia](https://numpy.org/doc/)

[Pandas dokumentácia](https://pandas.pydata.org/docs/)

[Typical Analysis Procedure](https://work.thaslwanter.at/Stats/html/statsAnalysis.html)


[Scikit-learn dokumentácia](https://scikit-learn.org/1.5/api/index.html)

[Feature importance](https://machinelearningmastery.com/calculate-feature-importance-with-python/)

[Scaling](https://machinelearningmastery.com/standardscaler-and-minmaxscaler-transforms-in-python/)

[Transformations](https://machinelearningmastery.com/power-transforms-with-scikit-learn/)